## Intro

In this notebook, I try to aggregating and averaging data per hour. The data used are **Tide Gauge** data, obtained from [TAD SERVER EUROPE](https://webcritech.jrc.ec.europa.eu/TAD_server/Device/483).

But, as you can see below, the interval of data measurement it's not consistent. As a result, we can't do the slicing method on pandas Data Frame for aggregating and averaging data. So, how we can tackle this problem?, let's check the code below.

In [8]:
# load dataset
import pandas as pd

data1 = pd.read_csv('/content/IDSL-305_data.csv')

print(data1.shape)
data1.head(10)

(5217, 11)


,Time(UTC),Lev RAD (m),Panel (V),rms (m),Sensor Temp (C),Temperature (C),Alert,Alert Signal,Battery (V),Forecast 30 (m),Forecast 300 (m)
0,01/08/2020 00:12:21,1.493,-5.5,0.014,49.4,38.228,0,0.021,12.127,1.492,1.470
1,01/08/2020 00:12:51,1.503,-5.5,0.014,49.4,38.609,0,0.023,12.132,1.492,1.469
2,01/08/2020 00:18:45,1.479,-5.5,0.014,49.9,38.471,0,0.033,12.140,1.482,1.449
3,01/08/2020 00:20:39,1.471,-5.5,0.014,49.9,38.495,0,0.034,12.187,1.477,1.443
4,01/08/2020 00:32:09,1.414,-5.5,0.014,49.9,39.084,0,0.010,12.173,1.411,1.401
5,01/08/2020 00:51:33,1.358,-5.5,0.015,49.9,40.111,0,0.023,12.391,1.361,1.338
6,01/08/2020 01:11:09,1.336,-5.5,0.015,50.5,42.167,0,0.067,12.623,1.338,1.271
7,01/08/2020 01:12:27,1.324,-5.5,0.015,50.5,41.838,0,0.064,12.746,1.332,1.268
8,01/08/2020 01:14:39,1.302,-5.5,0.015,50.5,42.063,0,0.048,12.732,1.310,1.262
9,01/08/2020 01:18:15,1.283,-5.5,0.015,50.5,42.448,0,0.034,12.667,1.285,1.252


As you can see in the table above, particularly on `Time (UTC)` column, the interval of data measurement it's not consistent. So, we can't do the slicing method for aggregating data.

To solve this problem, I split items on `Time (UTC)` column, into `date_hour`, `minute`, and `second`.

In [9]:
# Splitting items on 'Time (UTC) column' based on ":"
data1[['date_hour','minute','second']] = data1['Time(UTC)'].str.split(':', expand=True) 

data1.head(12)

,Time(UTC),Lev RAD (m),Panel (V),rms (m),Sensor Temp (C),Temperature (C),Alert,Alert Signal,Battery (V),Forecast 30 (m),Forecast 300 (m),date_hour,minute,second
0,01/08/2020 00:12:21,1.493,-5.5,0.014,49.4,38.228,0,0.021,12.127,1.492,1.470,01/08/2020 00,12,21
1,01/08/2020 00:12:51,1.503,-5.5,0.014,49.4,38.609,0,0.023,12.132,1.492,1.469,01/08/2020 00,12,51
2,01/08/2020 00:18:45,1.479,-5.5,0.014,49.9,38.471,0,0.033,12.140,1.482,1.449,01/08/2020 00,18,45
3,01/08/2020 00:20:39,1.471,-5.5,0.014,49.9,38.495,0,0.034,12.187,1.477,1.443,01/08/2020 00,20,39
4,01/08/2020 00:32:09,1.414,-5.5,0.014,49.9,39.084,0,0.010,12.173,1.411,1.401,01/08/2020 00,32,09
5,01/08/2020 00:51:33,1.358,-5.5,0.015,49.9,40.111,0,0.023,12.391,1.361,1.338,01/08/2020 00,51,33
6,01/08/2020 01:11:09,1.336,-5.5,0.015,50.5,42.167,0,0.067,12.623,1.338,1.271,01/08/2020 01,11,09
7,01/08/2020 01:12:27,1.324,-5.5,0.015,50.5,41.838,0,0.064,12.746,1.332,1.268,01/08/2020 01,12,27
8,01/08/2020 01:14:39,1.302,-5.5,0.015,50.5,42.063,0,0.048,12.732,1.310,1.262,01/08/2020 01,14,39
9,01/08/2020 01:18:15,1.283,-5.5,0.015,50.5,42.448,0,0.034,12.667,1.285,1.252,01/08/2020 01,18,15


### Drop unnecessary columns

In [10]:
data1.drop(['minute','second'], axis=1, inplace=True)
data1.head()

,Time(UTC),Lev RAD (m),Panel (V),rms (m),Sensor Temp (C),Temperature (C),Alert,Alert Signal,Battery (V),Forecast 30 (m),Forecast 300 (m),date_hour
0,01/08/2020 00:12:21,1.493,-5.5,0.014,49.4,38.228,0,0.021,12.127,1.492,1.470,01/08/2020 00
1,01/08/2020 00:12:51,1.503,-5.5,0.014,49.4,38.609,0,0.023,12.132,1.492,1.469,01/08/2020 00
2,01/08/2020 00:18:45,1.479,-5.5,0.014,49.9,38.471,0,0.033,12.140,1.482,1.449,01/08/2020 00
3,01/08/2020 00:20:39,1.471,-5.5,0.014,49.9,38.495,0,0.034,12.187,1.477,1.443,01/08/2020 00
4,01/08/2020 00:32:09,1.414,-5.5,0.014,49.9,39.084,0,0.010,12.173,1.411,1.401,01/08/2020 00


### Aggregating data per 1 hour interval

In [11]:
data1_per_interval_1jam = pd.pivot_table(data1, index='date_hour', aggfunc='first')

print(data1_per_interval_1jam.shape)
data1_per_interval_1jam.head()

(701, 11)


,Alert,Alert Signal,Battery (V),Forecast 30 (m),Forecast 300 (m),Lev RAD (m),Panel (V),Sensor Temp (C),Temperature (C),Time(UTC),rms (m)
date_hour,,,,,,,,,,,
01/08/2020 00,0,0.021,12.127,1.492,1.470,1.493,-5.5,49.4,38.228,01/08/2020 00:12:21,0.014
01/08/2020 01,0,0.067,12.623,1.338,1.271,1.336,-5.5,50.5,42.167,01/08/2020 01:11:09,0.015
01/08/2020 02,0,0.036,13.264,1.140,1.104,1.132,-5.5,52.6,48.109,01/08/2020 02:03:33,0.018
01/08/2020 03,0,0.030,13.190,0.965,0.935,0.957,-5.5,56.9,61.535,01/08/2020 03:07:57,0.022
01/08/2020 04,0,0.030,13.087,0.894,0.863,0.895,-5.5,60.1,69.910,01/08/2020 04:07:31,0.022


### Averaging data per 1 hour

In [12]:
import numpy as np

# average per one hour
data1_per_jam = pd.pivot_table(data1, index = ['date_hour'], aggfunc = np.mean)
data1_per_jam

,Alert,Alert Signal,Battery (V),Forecast 30 (m),Forecast 300 (m),Lev RAD (m),Panel (V),Sensor Temp (C),Temperature (C),rms (m)
date_hour,,,,,,,,,,
01/08/2020 00,0.0,0.024000,12.191667,1.452500,1.428333,1.453000,-5.5,49.733333,38.833000,0.014167
01/08/2020 01,0.0,0.034583,12.796667,1.249083,1.214417,1.243083,-5.5,50.916667,43.899417,0.015083
01/08/2020 02,0.0,0.030000,13.254000,1.059500,1.029333,1.055333,-5.5,54.416667,53.780333,0.019667
01/08/2020 03,0.0,0.042625,13.123375,0.941875,0.899375,0.941875,-5.5,58.550000,66.362250,0.020625
01/08/2020 04,0.0,0.042111,12.539667,0.911000,0.868778,0.912000,-5.5,59.877778,69.104556,0.020556
...,...,...,...,...,...,...,...,...,...,...
31/08/2020 19,0.0,0.012875,11.996375,1.185625,1.192500,1.188875,-5.5,50.600000,40.488250,0.012750
31/08/2020 20,0.0,0.016625,11.984250,1.397375,1.397250,1.396125,-5.5,50.275000,40.310000,0.010875
31/08/2020 21,0.0,0.007000,11.959000,1.616000,1.621400,1.616200,-5.5,50.380000,40.097500,0.015500


### Save files to excel

In [ ]:
# menyimpan file
data1_per_interval_1jam.to_excel('/content/IDSL-305_data_interval_1jam.xlsx')
data1_ave_per_jam.to_excel('/content/IDSL-305_data_per_jam.xlsx')